In [1]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns

Load Data

In [2]:
raw_df = pd.read_csv(r'C:\Users\chaiyaphop.jam\Documents\Works\Markov_Chains\TEST_T_DATA_SET_599.csv')
raw_df.head()

,CUSTOMER_ID,SUBSCRIBER,PRODUCT,PACKAGE_01,PACKAGE_02,PACKAGE_03,PACKAGE_04,PACKAGE_05,PACKAGE_06
0,1,11,T,599,699,799,899,999,599
1,2,22,T,599,699,799,899,999,1099
2,3,33,T,599,699,799,899,999,1099
3,4,44,T,599,699,799,899,999,1099
4,5,55,T,599,699,799,899,999,1099


Select Features

In [3]:
df = raw_df[['PACKAGE_01','PACKAGE_02','PACKAGE_03','PACKAGE_04','PACKAGE_05','PACKAGE_06']]
df.describe().round(2)

,PACKAGE_01,PACKAGE_02,PACKAGE_03,PACKAGE_04,PACKAGE_05,PACKAGE_06
count,3950.0,3950.00,3950.00,3950.00,3950.00,3950.00
mean,599.0,749.03,849.03,949.03,1049.03,1048.75
std,0.0,50.01,50.01,50.01,50.01,51.01
min,599.0,699.00,799.00,899.00,999.00,599.00
25%,599.0,699.00,799.00,899.00,999.00,999.00
50%,599.0,799.00,899.00,999.00,1099.00,999.00
75%,599.0,799.00,899.00,999.00,1099.00,1099.00
max,599.0,799.00,899.00,999.00,1099.00,1099.00


Remove Outlier

In [4]:
# outlier_p1 = df.PACKAGE_01 > df.PACKAGE_01.quantile(1)
# outlier_p2 = df.PACKAGE_02 > df.PACKAGE_02.quantile(1)
# outlier_p3 = df.PACKAGE_03 > df.PACKAGE_03.quantile(1)
# outlier_p4 = df.PACKAGE_04 > df.PACKAGE_04.quantile(1)
# outlier_p5 = df.PACKAGE_05 > df.PACKAGE_05.quantile(1)
# outlier_p6 = df.PACKAGE_06 > df.PACKAGE_06.quantile(1)
# outlier_p1 = df.PACKAGE_01 > df.PACKAGE_01.quantile(1)
# outlier_p2 = df.PACKAGE_02 > df.PACKAGE_02.quantile(1)
# outlier_p3 = df.PACKAGE_03 > df.PACKAGE_03.quantile(1)
# outlier_p4 = df.PACKAGE_04 > df.PACKAGE_04.quantile(1)
# outlier_p5 = df.PACKAGE_05 > df.PACKAGE_05.quantile(1)
# outlier_p6 = df.PACKAGE_06 > df.PACKAGE_06.quantile(1)

# flag_outlier = outlier_p1 | outlier_p2 | outlier_p3 | outlier_p4 | outlier_p5 | outlier_p6

# df_clean = df.loc[~flag_outlier]
# print(df_clean.describe().round(2))
# print(df_clean.corr())

Scatter Plot

In [5]:
# sns.pairplot(df_clean, diag_kind = 'kde')

Sample Data

In [6]:
df_array = np.array(df)
df_array[:5]

array([[ 599,  699,  799,  899,  999,  599],
       [ 599,  699,  799,  899,  999, 1099],
       [ 599,  699,  799,  899,  999, 1099],
       [ 599,  699,  799,  899,  999, 1099],
       [ 599,  699,  799,  899,  999, 1099]], dtype=int64)

Possible State and Encode Possible State

In [7]:
all_state = []
for row in df_array:
    for col in row:
        all_state.append(col)

possible_state = np.sort(np.unique(all_state)).tolist()
print(f'Possible State is {possible_state}')

encoded_possible_state = []
for state in possible_state:
    for i in range(len(possible_state)):
        if possible_state.index(state) == i:
            encoded_possible_state.append(i)
print(f'Encoded Possible State is {encoded_possible_state}')

n_possible_state = 0
for i in encoded_possible_state:
    n_possible_state += 1
print(f'Number of Possible State is {n_possible_state}')

Possible State is [599, 699, 799, 899, 999, 1099]
Encoded Possible State is [0, 1, 2, 3, 4, 5]
Number of Possible State is 6


Encode Features

In [8]:
for i in possible_state:
    np.place(df_array, df_array == i, [possible_state.index(i)])
df_train = df_array
df_train[:5]

array([[0, 1, 2, 3, 4, 0],
       [0, 1, 2, 3, 4, 5],
       [0, 1, 2, 3, 4, 5],
       [0, 1, 2, 3, 4, 5],
       [0, 1, 2, 3, 4, 5]], dtype=int64)

Markov Chain Model

In [9]:
# Estimate Transition function
def transition_prob_estimator(df_train):
    est_trans_prob = np.zeros((n_possible_state,n_possible_state))

    for row in range(est_trans_prob.shape[0]): # previose state
        for col in range(est_trans_prob.shape[1]): # future state
            target_transition = [row,col]

            # converting 2 time steps for each seq
            for each_step in range(1,len(df_train[0])):
                this_transition = np.array(df_train)[:,[each_step-1,each_step]].tolist()

                for each_seq in this_transition:
                    if each_seq == target_transition:
                        est_trans_prob[row,col] += 1
            
            # sum up to 1
            est_trans_prob[row,col] /= len(np.argwhere(np.array(df_train)[:,:-1] == row))
    
    return est_trans_prob.round(2)

# predictive function
def prediction(est_trans_prob, train_seq):
    next_state_list = []
    for each_seq in range(pd.DataFrame(df_train).count()[1]):
        current_state = train_seq[each_seq][-1]

        next_state = np.argmax(est_trans_prob[current_state,:])
        next_state_list.append(next_state)
        
    return next_state_list

In [10]:
# Train Test Split
train_seq = np.array(df_train)[:,:-1]
test_seq = np.array(df_train)[:,-1]

# Transition Prob. Matrix Estimation
est_trans_prob = transition_prob_estimator(df_train)
print('Transition Prob. Matrix')
print(pd.DataFrame(est_trans_prob))

# Prediction
pred_seq = prediction(est_trans_prob, train_seq)

# Accuracy
print('%Accuracy :',round((pred_seq == test_seq).sum()/len(test_seq)*100),'%')

Transition Prob. Matrix
     0    1    2    3    4    5
0  0.0  0.5  0.5  0.0  0.0  0.0
1  0.0  0.0  1.0  0.0  0.0  0.0
2  0.0  0.0  0.0  1.0  0.0  0.0
3  0.0  0.0  0.0  0.0  1.0  0.0
4  0.0  0.0  0.0  0.0  0.0  1.0
5  0.0  0.0  0.0  0.0  1.0  0.0
%Accuracy : 100 %


Prediction

In [11]:
# Actual Data Set
current_state = np.array(df_array)
last_state = np.array(df_array)[:,-1]

# Prediction
future_state = prediction(est_trans_prob, current_state)

# %Change
print('%Change from Last State :',round((1-((future_state == last_state).sum()/len(last_state)))*100),'%')

%Change from Last State : 100 %


Export

In [12]:
# Decode Future State
decode_future_state = []

for i in future_state:
    if i == possible_state.index(possible_state[i]):
        decode_future_state.append(possible_state[i])
future_state_df = pd.DataFrame(decode_future_state)
future_state_df.columns = ['PRED_PACKAGE']

result = pd.concat([raw_df,future_state_df], axis=1)

# Export
export_result = result.to_csv(r'C:\Users\chaiyaphop.jam\Desktop\Result_Markov.csv')
result

,CUSTOMER_ID,SUBSCRIBER,PRODUCT,PACKAGE_01,PACKAGE_02,PACKAGE_03,PACKAGE_04,PACKAGE_05,PACKAGE_06,PRED_PACKAGE
0,1,11,T,599,699,799,899,999,599,699
1,2,22,T,599,699,799,899,999,1099,999
2,3,33,T,599,699,799,899,999,1099,999
3,4,44,T,599,699,799,899,999,1099,999
4,5,55,T,599,699,799,899,999,1099,999
...,...,...,...,...,...,...,...,...,...,...
3945,3946,39463946,T,599,799,899,999,1099,999,1099
3946,3947,39473947,T,599,799,899,999,1099,999,1099
3947,3948,39483948,T,599,799,899,999,1099,999,1099
3948,3949,39493949,T,599,799,899,999,1099,999,1099
